In [235]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [236]:
# Carga de tabla con explicaciones de cada columna

col_explicacion = pd.read_csv('descriptions_for_ column.csv', encoding='latin-1')
col_explicacion = col_explicacion.drop(columns=['Col_names_english', 
                                                'Descriptions_english'])
print(col_explicacion)

      Col_names_spanish                               Descriptions_spanish
0     Fecha de Registro                     Fecha de registro del divorcio
1                  Tipo  Necesario cuando alguna de las partes no acept...
2          Nacionalidad                        Nacionalidad del divorciado
3             F_Naci_Do                 Fecha de nacimiento del divorciado
4            LocNaci_Do             Localidad de nacimiento del divorciado
5           MpioNaci_Do             Municipio de nacimiento del divorciado
6           EFedNaci_Do    Entidad Federativa de nacimiento del divorciado
7           PaisNaci_Do                 PaÃ­s de nacimiento del divorciado
8                  Edad                                Edad del divorciado
9            MpioHab_Do   Municipio del domicilio que habita el divorciado
10           EFedHab_Do  Entidad federativa del domicilio que habita el...
11           PaisHab_Do    PaÃ­s del domicilio en que habita el divorciado
12            ElIngreso  

In [237]:
# Carga de DF + renombre de columnas + descarte de columnas 

df = pd.read_csv('divorces_2000-2015_original.csv', 
                 parse_dates=['Fecha de Registro', 'F_Naci_Do', 'F_Naci_Da', 'Fecha'],
                 date_format='%d/%m/%y')

df = df.rename(columns={'Ocupaci√≥n_Do': 'Ocupacion_Do', 
                   'Ocupaci√≥n_Da': 'Ocupacion_Da',
                   'Matri_A√±os': 'Matri_Años',
                   'Edad': 'Edad_Do', 'Edad2': 'Edad_Da',
                   'Nacionalidad': 'Nacionalidad_Do', 
                   'Nacionalidad2': 'Nacionalidad_Da',
                   'Fecha': 'Fecha_matri', 'Fecha de Registro': 'Fecha_divor'})

df = df.drop(columns=['LocNaci_Do', 'MpioNaci_Do', 'EFedNaci_Do', 'MpioHab_Do',
                      'EFedHab_Do', 'LocHabi_Do', 'LocNaci_Da', 'MpioNaci_Da', 
                      'EFedNaci_Da', 'MpioHab_Da','EFedHab_Da', 'LocHabi_Da',
                      'Fecha de Registro 2', 'Localidad', 'Municipio', 
                      'Matri_Meses'])

display(df.columns)

Index(['Fecha_divor', 'Tipo', 'Nacionalidad_Do', 'F_Naci_Do', 'PaisNaci_Do',
       'Edad_Do', 'PaisHab_Do', 'ElIngreso', 'Ocupacion_Do', 'Nacionalidad_Da',
       'F_Naci_Da', 'PaisNaci_Da', 'Edad_Da', 'PaisHab_Da', 'Ocupacion_Da',
       'EllaIngreso', 'Fecha_matri', 'Entidad Federativa', 'ElEscolaridad',
       'ElTrabajo', 'EllaEscolaridad', 'EllaTrabajo', 'Matri_Años',
       'Num_Hijos', 'Custodia'],
      dtype='object')

In [238]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4923 entries, 0 to 4922
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Fecha_divor         4923 non-null   datetime64[ns]
 1   Tipo                4923 non-null   object        
 2   Nacionalidad_Do     4922 non-null   object        
 3   F_Naci_Do           4542 non-null   datetime64[ns]
 4   PaisNaci_Do         4796 non-null   object        
 5   Edad_Do             4816 non-null   float64       
 6   PaisHab_Do          4599 non-null   object        
 7   ElIngreso           3504 non-null   float64       
 8   Ocupacion_Do        4394 non-null   object        
 9   Nacionalidad_Da     4920 non-null   object        
 10  F_Naci_Da           4471 non-null   datetime64[ns]
 11  PaisNaci_Da         4784 non-null   object        
 12  Edad_Da             4772 non-null   float64       
 13  PaisHab_Da          4618 non-null   object      

In [240]:
df.isnull().sum()

Fecha_divor           0
Tipo                  0
Nacionalidad_Do       0
F_Naci_Do             0
PaisNaci_Do           0
Edad_Do               0
PaisHab_Do            0
ElIngreso             0
Ocupacion_Do          0
Nacionalidad_Da       0
F_Naci_Da             0
PaisNaci_Da           0
Edad_Da               0
PaisHab_Da            0
Ocupacion_Da          0
EllaIngreso           0
Fecha_matri           0
Entidad Federativa    0
ElEscolaridad         0
ElTrabajo             0
EllaEscolaridad       0
EllaTrabajo           0
Matri_Años            0
Num_Hijos             0
Custodia              0
dtype: int64

In [ ]:
# Manejo de Nulos Cualitativos (llenados con la moda)
col_cualitativas = ['PaisNaci_Do', 'PaisNaci_Da', 'Nacionalidad_Do',
                    'Nacionalidad_Da', 'PaisHab_Do', 'PaisHab_Da',
                    'Ocupacion_Do', 'Ocupacion_Da', 'ElEscolaridad',
                    'EllaEscolaridad', 'ElTrabajo', 'EllaTrabajo', 'Custodia',
                    'F_Naci_Do', 'F_Naci_Da']

for col in col_cualitativas:
    df[col] = df[col].fillna(df[col].mode()[0])

# Manejo de Nulos Cuantitativos (llenados con la mediana)
col_cuantitativas = ['Edad_Do', 'Edad_Da', 'ElIngreso', 'EllaIngreso']

for col in col_cuantitativas:
    df[col] = df[col].fillna(df[col].median())
    
# Manejo de Años de Matrimonio nulos, se calculan con
# (fecha de divorcio - fecha de matrimonio) redondeados sin decimales
dif_dias = (df['Fecha_divor'] - df['Fecha_matri']).dt.days
years_casados = round(dif_dias / 365.25, 0) #.25 para considerar años bisiestos
df['Matri_Años'] = df['Matri_Años'].fillna(years_casados)

# Se asumen que los NaN en Num_hijos quieren decir 0 hijos (mas del 40% de filas)
df['Num_Hijos'] = df['Num_Hijos'].fillna(0)